# Setup

In [38]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
try:
    #import google.colab # type: ignore
    #from google.colab import output
    %pip install sae-lens transformer-lens
except:
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

In [3]:
from transformer_lens import HookedTransformer
from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner

from torch import nn, Tensor
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

from datasets import load_dataset
from rich import print as rprint  # printing highlights in data samples
import pickle
from google.colab import files

import torch
import os

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using device: cuda


# change input params here

In [75]:
model_name = "tiny-stories-1L-21M"
# model_name = "tiny-stories-2L-33M"
short_model_name = 'ts_1L_21M'
# short_model_name = 'ts_2L_33M'
layer_name = "blocks.0.hook_mlp_out"
hook_layer = 0
d_in = 1024
expa_fac = 8  # 8, 16, 32
total_training_steps = 100000 # 30_000
activation_fn_str = "topk"
activation_fn_kwargs = {"k":32}

# wandb_project = "sae_" + model_name+"_MLP" + str(hook_layer) + "_df-" + str(d_in * expa_fac)
wandb_project = "sae_" + model_name + "_MLP" + str(hook_layer) + "_df" + str(d_in * expa_fac) + "_steps100k" + "_topK"
drive_save_path = '/content/drive/MyDrive/'+wandb_project+'.pth'

save_data_fn = 'batch_tokens_anySamps_v1.pkl'
# Wdec_filename = 'Wdec_' + short_model_name + "_df" + str(d_in * expa_fac) + '.pkl'
# acts_save_path = 'fActs_' + short_model_name + "_df_" + str(d_in * expa_fac)  + '_anySamps_v1.pkl'
Wdec_filename = 'Wdec_' + short_model_name + "_df" + str(d_in * expa_fac)  + "_steps100k" + "_topK" + '.pkl'
acts_save_path = 'fActs_' + short_model_name + "_df_" + str(d_in * expa_fac)  + "_steps100k" + "_topK" + '_anySamps_v1.pkl'

# load model

In [57]:
model = HookedTransformer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


# load dataset

## get data

Need load model tokenizer before obtain dataset

In [ ]:
from transformer_lens.utils import tokenize_and_concatenate

dataset = load_dataset("roneneldan/TinyStories", streaming=False)
test_dataset = dataset['validation']

token_dataset = tokenize_and_concatenate(
    dataset = test_dataset,
    tokenizer = model.tokenizer, # type: ignore
    streaming=True,
    # max_length=sae.cfg.context_size,
    max_length=128,
    # add_bos_token=sae.cfg.prepend_bos,
    add_bos_token=False,
)

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

Map:   0%|          | 0/21990 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (10434 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
batch_tokens = token_dataset[:500]["tokens"]
batch_tokens.shape

torch.Size([500, 128])

## save selected data

In [ ]:
with open(save_data_fn, 'wb') as f:
    pickle.dump(batch_tokens, f)

# source_path = f'/path/to/your/file/{file_name}'
source_path = save_data_fn
# dest_folder = ''
destination_path = f'/content/drive/MyDrive/{save_data_fn}'

shutil.copy(source_path, destination_path) # Copy the file

## load selected data

In [6]:
# check if saved
file_path = '/content/drive/MyDrive/' + save_data_fn
with open(file_path, 'rb') as f:
    batch_tokens = pickle.load(f)

# load sae using saelens

## load

To get feature actvs, use sae_lens class method. To do this, you must load the sae as the sae class (wrapper over torch model).

In [76]:
batch_size = 4096
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name=model_name,  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    hook_name=layer_name,  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_layer=hook_layer,  # Only one layer in the model.
    d_in=d_in,  # the width of the mlp output.
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    streaming=True,  # we could pre-download the token dataset if it was small.
    # SAE Parameters
    activation_fn = activation_fn_str,
    activation_fn_kwargs = activation_fn_kwargs,
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor= expa_fac,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    scale_sparsity_penalty_by_decoder_norm=True,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_activations="expected_average_only_in",
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size_tokens=batch_size,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size_prompts=16,
    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project=wandb_project,
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32"
)

Run name: 8192-L1-5-LR-5e-05-Tokens-4.096e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 100000
Total wandb updates: 3333
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 100 times.
Number tokens in sparsity calculation window: 4.10e+06


In [77]:
# https://github.com/jbloomAus/SAELens/blob/main/sae_lens/sae.py#L13
from sae_lens import SAEConfig

cfg_dict = cfg.to_dict()
cfg_dict['finetuning_scaling_factor'] = 0

sae_cfg = SAEConfig.from_dict(cfg_dict)

from sae_lens import SAE
sae = SAE(sae_cfg)

state_dict = torch.load(drive_save_path)
sae.load_state_dict(state_dict)

<All keys matched successfully>

## save decoder weights

In [78]:
weight_matrix_np = sae.W_dec.cpu()

In [79]:
with open(Wdec_filename, 'wb') as f:
    pickle.dump(weight_matrix_np, f)

# source_path = f'/path/to/your/file/{file_name}'
source_path = Wdec_filename
# dest_folder = ''
destination_path = f'/content/drive/MyDrive/{Wdec_filename}'

shutil.copy(source_path, destination_path) # Copy the file

'/content/drive/MyDrive/Wdec_ts_1L_21M_df8192_steps100k_topK.pkl'

# save top ds exm for features

## get LLM actvs

In [80]:
h_store = torch.zeros((batch_tokens.shape[0], batch_tokens.shape[1], model.cfg.d_model), device=model.cfg.device)
h_store.shape

torch.Size([500, 128, 1024])

In [81]:
def store_h_hook(
    pattern: Float[Tensor, "batch seqlen d_model"],
    hook
):
    h_store[:] = pattern  # this works b/c changes values, not replaces entire thing

In [82]:
model.run_with_hooks(
    batch_tokens,
    return_type = None,
    fwd_hooks=[
        (layer_name, store_h_hook),
    ]
)

## get SAE actvs

In [83]:
sae.eval()  # prevents error if we're expecting a dead neuron mask for who grads
with torch.no_grad():
    feature_acts = sae.encode(h_store)

In [84]:
def count_nonzero_features(feature_acts):
    # Count the number of 0s in the tensor
    # num_zeros = (feature_acts == 0).sum().item() # OOM
    # chunk
    num_zeros = 0
    for i in range(feature_acts.size(0)):
        for j in range(feature_acts.size(1)):
            num_zeros += (feature_acts[i, j, :] == 0).sum().item()

    # Count the number of nonzeroes in the tensor
    # num_ones = (feature_acts > 0).sum().item()
    num_ones = 0
    for i in range(feature_acts.size(0)):
        for j in range(feature_acts.size(1)):
            num_ones += (feature_acts[i, j, :] > 0).sum().item()

    # Calculate the percentage of 1s over 0s
    if num_zeros > 0:
        perc_ones_over_total = (num_ones / (num_ones + num_zeros)) * 100
    else:
        perc_ones_over_total = float('inf')  # Handle division by zero
    return perc_ones_over_total
count_nonzero_features(feature_acts)

0.21240043640136716

## save actvs

Now you have to save actvs, bc saelens not compatible with umap OR cosine sim lib

In [85]:
with open(acts_save_path, 'wb') as f:
    pickle.dump(feature_acts, f)

# source_path = f'/path/to/your/file/{file_name}'
source_path = acts_save_path
# dest_folder = ''
destination_path = f'/content/drive/MyDrive/{acts_save_path}'

shutil.copy(source_path, destination_path) # Copy the file

'/content/drive/MyDrive/fActs_ts_1L_21M_df_8192_steps100k_topK_anySamps_v1.pkl'

In [86]:
# check if saved
file_path = '/content/drive/MyDrive/' + acts_save_path
with open(file_path, 'rb') as f:
    feature_acts = pickle.load(f)

## interpret by dataset examples, keyword search

In [87]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
) -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [88]:
def store_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        s += f'batchID: {batch_idx}, '
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        for i in range(seq_start, seq_end):
            new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'
        feat_samps.append(seq)
    return s, feat_samps

In [89]:
# store feature : str of top 5 snippets (contains batchID, val, seq in each substr)
feat_snip_dict_strs = {}
feat_snip_dict_lst = {}
samp_m = 5

for feature_idx in range(feature_acts.shape[-1]):
    if feature_idx % 1000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts, feature_idx, samp_m, batch_tokens=batch_tokens)
    disp_str, feat_samps = store_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens)
    feat_snip_dict_strs[feature_idx] = disp_str
    feat_snip_dict_lst[feature_idx] = feat_samps

Feature:  0
Feature:  1000
Feature:  2000
Feature:  3000
Feature:  4000
Feature:  5000
Feature:  6000
Feature:  7000
Feature:  8000


In [90]:
relevant_features = {}

# keyword = 'princess'
keyword = '[bold u dark_orange] princess[/]'
relevant_features[keyword] = []
for f_id, str_output in feat_snip_dict_strs.items():
    if keyword in str_output:
        relevant_features[keyword].append(f_id)
        print(f'Feature: {f_id}')
        rprint(str_output)

Feature: 2479


batchID: 71, Act = 3.34, Seq = ". \n\nThe lion saw a man with"
batchID: 453, Act = 3.32, Seq = " named Timmy. Timmy loved going to the"
batchID: 188, Act = 3.31, Seq = "." \n\nEllie smiled and said"
batchID: 108, Act = 3.26, Seq = ". \n\nThe judge noticed Lily and Max"
batchID: 70, Act = 3.24, Seq = " anymore.\n\nThe princess felt bad and wanted"

Feature: 2658


batchID: 361, Act = 9.83, Seq = " You are a very nice princess."
batchID: 391, Act = 9.07, Seq = "". He was a good boy.Tom and Lily"
batchID: 188, Act = 7.53, Seq = " You are such a great teacher." \n\n"
batchID: 495, Act = 7.44, Seq = "! You are a good friend." Lily smiled and"
batchID: 279, Act = 6.42, Seq = " him he was a good boy for protecting his house"

Feature: 3452


batchID: 374, Act = 7.38, Seq = " a story about a brave knight and a friendly dragon"
batchID: 282, Act = 6.49, Seq = ". Maybe it is a monster cloud!" Anna said"
batchID: 129, Act = 6.46, Seq = " and pretend to be a princess. One day,"
batchID: 65, Act = 6.43, Seq = " dreamt that a big monster came out of my"
batchID: 226, Act = 6.40, Seq = " want to be a fireman like you when I"

## save datasamps for features

In [91]:
with open('feat_snip_dict_strs.pkl', 'wb') as f:
    pickle.dump(feat_snip_dict_strs, f)
files.download('feat_snip_dict_strs.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [92]:
import json

# Transform the dictionary into a list of dictionaries
feature_top_samps_lst = [{"feature": feature, "strings": strings} for feature, strings in feat_snip_dict_lst.items()]

# Save to a JSON file
output_file = 'feature_top_samps_lst.json'
with open(output_file, 'w') as f:
    json.dump(feature_top_samps_lst, f, indent=4)

files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>